# Homework 5 - Group 19
## Visit the Wikipedia hyperlinks graph!

**Group's members:** Francesco Russo, Marco Vicentini & Sara Cordaro


### Research questions 1


At the beginning we imported all the libraries and declared the global variables.

In [1]:
# libraries
import pandas as pd
from collections import defaultdict
import networkx as nx
import numpy as np
import pickle
from tqdm import tqdm
import heapq

In [2]:
# global variables
# Hyperlink network of Wikipedia
hyperlink = 'wiki-topcats-reduced.txt'
# Which articles are in which of the top categories
categories = 'wiki-topcats-categories.txt'
# Names of the articles
articles = 'wiki-topcats-page-names.txt'

Here we worked with the file *wiki-topcats-page-names.txt*, that associate an index to each article.

We decided to create a dictionary that has the article's id as key and the article's name as value. Then we converted the dictionary in a dataframe to check if the names of each article is correct, without escape or other particular characters.

In [3]:
# create dictionary with article's id as key and article's name as value
idx_name = {}
with open(articles) as f:
    for line in f:
        (key, val) = line.split(' ', 1)
        idx_name[int(key)] = val.strip()
        

In [4]:
# convert dictionary in datframe
df_articles = pd.DataFrame.from_dict(idx_name, orient='index', columns = ['name'])

In [5]:
df_articles.head()

,name
0,Chiasmal syndrome
1,Kleroterion
2,Pinakion
3,LyndonHochschildSerre spectral sequence
4,Zariski's main theorem


Now we create our **graph**. 

We worked with the file *wiki-topcats-reduced.txt*, that show us the edges of each vertex.

In [6]:
# create dictionary with article's id as key and list of article's hyperlinks as value
# so we can take only the nodes requested
graph = defaultdict(list)
with open(hyperlink) as f:
    for line in f:
        (key, val) = line.split('\t')
        graph[int(key)].append(int(val.strip()))

The variable *vertices_n* is the number of all articles find in the first dictionary *df_articles*, **1791489**.

In [7]:
# number of vertices
vertices_n = len(idx_name.keys())
print(vertices_n)

1791489


The variable *nodes_wc* is the number of vertices pf the *graph* that has at least a outgoing edge, **428957**.

In [8]:
# number of vertices without some categories removed from the graph
nodes_wc = len(graph.keys())
print(nodes_wc)

428957


We used the library *networkx* to try to plot the graph, but it is too big.


In these code's lines we computed the total number of edges **2645247**, saved in the variable *edges_n*.

In [9]:
# intialize directed graph
graph_nx = nx.DiGraph()
graph_nx.add_nodes_from(list(graph.keys()))
# add edges
edges_n = 0
for k in graph.keys():
    edges_n += len(graph[k])
    for j in graph[k]:
        graph_nx.add_edge(k,j)

In [10]:
# number of edges
print(edges_n)

2645247


28511807 is the edges's number of the total graph, considering all the categories.


To be more precise we calculated the total number of nodes in the graph, including the nodes with only incident edges.
We found **461193** vertices.

In [11]:
# all number of vertices without some categories removed from the graph 
# also the nodes that has only incident edges
all_nodes = list(graph.keys())
for k in graph.keys():
    all_nodes += graph[k]
all_nodes = set(all_nodes)

In [12]:
all_nodes_wc = len(all_nodes)

In [13]:
print(all_nodes_wc)

461193


Here we check if the graph is directed or undirected, so we studied the graph. 

- We iterate all the nodes;
- We take the node *A*;
- We take the list of vertices, connected to the node *A* that we are checking;
- We take all the nodes in the list and we take the node *B*; 
- We check if the node *A* is present in the list of vertices, connected to the node *B*;
- If we find just a node *B* that doesn't have the node *A* in its list, we can say that the graph is directed, so we stop every loop.

In [14]:
# check if graph is directed or not
directed = False
for k in graph.keys():
    for j in graph[k]:
        if k not in graph[j]:
            directed = True
        break
    break
if directed == True:
    print('The graph is directed')
else:
    print('The graph is undirected')

The graph is directed


The result of this function says that the graph is **directed**.


At this point we check if the graph is **dense** or **sparse**.

A **dense** graph is a graph in which the number of edges is close to the maximal number of edges. The opposite, a graph with only a few edges, is a **sparse** graph. The distinction between sparse and dense graphs is rather vague, and depends on the context.


For **undirected simple** graphs, the graph density is defined as:

$D={\frac {2|E|}{|V|\,(|V|-1)}}$

For **directed simple** graphs, the graph density is defined as:

$D={\frac {|E|}{|V|\,(|V|-1)}}$

where *E* is the number of edges and *V* is the number of vertices in the graph. The maximum number of edges for an undirected graph is $|V|(|V|-1)/2$, so the *maximal density* is 1 (for complete graphs) and the *minimal density* is 0. The maximum number of edges for a directed graph is $|V|(|V|-1)$

In [15]:
# maximum number of edges
max_edges_directed = abs(all_nodes_wc)*(abs(all_nodes_wc) - 1)

In [16]:
print(max_edges_directed)

212698522056


In [17]:
# graph density for a directed simple graphs
density_direct = (abs(edges_n))/max_edges_directed

In [18]:
print(density_direct)

1.2436602635647606e-05


The graph is **sparse**, because the density is 1.2436602635647606e-05.

**Handshaking Lemma** : $\sum _{v\in V}\deg(v)=2|E|\,$ total number of edges, if the graph is full.

Now to compute the degree of each vertex, we create a new dictionary *idx_degree*,  has the node as key and a list of incident edges as value. So we could count the number of incident edges for every node.

We found that the **average degree** is **7.503863632495362**.

In [18]:
# dictionary to compute the degree of each vertex
# idx_degree has the node as key and a list of incident edges as value
idx_degree = defaultdict(list)
for k in graph.keys():
    for j in graph[k]:
        idx_degree[j].append(k)

In [19]:
# average node degree
average_degree = 0
for k in idx_degree.keys():
    average_degree += len(idx_degree[k])
average_degree = average_degree/len(idx_degree.keys())    

In [20]:
print(average_degree)

7.503863632495362


In [21]:
# To plot the graph
# nx.draw_spectral(graph,with_labels = True,node_color = 'lightskyblue')

### Research questions 2

For the second research we worked with the file *wiki-topcats-categories.txt*, so we created a dictionary *category_art* with article's category as key and list of article's idx as value.

In [22]:
# create dictionary with article's category as key and list of article's idx as value
category_art = {}
count = 0
with open(categories) as f:
    for line in f:
        count += 1
        (key, val) = line.split(';')
        if val.strip() == '':
            category_art[key.replace('Category:','')] = []
        else:
            list_art = (val.strip()).split(' ')
            category_art[key.replace('Category:','')] = list(map(int,list_art))

Total number of all categories.

In [23]:
print(len(category_art.keys()))

17364


Here we took only the category with at least *3500* article, then we cleaned the dictionary and we took only the nodes belonging to the graph that we are studying.

In [24]:
# clean dictionary without some categories removed from the graph
category_clean = defaultdict(list)
for k in category_art.keys():
    # we take the categoy only if the list has at least 3500 category
    if len(category_art[k]) >= 3500:
        for node in category_art[k]:
            if node in all_nodes:
                category_clean[k].append(node)

Number and list of the categories that we have to consider.

In [25]:
# list of all categories
all_categories = list(category_clean.keys())
print(len(all_categories))

35


In [26]:
print(list(category_clean.keys()))

['English_footballers', 'The_Football_League_players', 'Association_football_forwards', 'Association_football_goalkeepers', 'Association_football_midfielders', 'Association_football_defenders', 'Living_people', 'Year_of_birth_unknown', 'Harvard_University_alumni', 'Major_League_Baseball_pitchers', 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 'Indian_films', 'Year_of_death_missing', 'English_cricketers', 'Year_of_birth_missing_(living_people)', 'Rivers_of_Romania', 'Main_Belt_asteroids', 'Asteroids_named_for_people', 'English-language_albums', 'English_television_actors', 'British_films', 'English-language_films', 'American_films', 'Fellows_of_the_Royal_Society', 'People_from_New_York_City', 'American_Jews', 'American_television_actors', 'American_film_actors', 'Debut_albums', 'Black-and-white_films', 'Year_of_birth_missing', 'Place_of_birth_missing_(living_people)', 'Article_Feedback_Pilot', 'American_military_personnel_of_World_War_II', 'Windows_games']


In this code's lines we ask a category $C_0 =\{article_1, article_2, \dots\}$ as input. Then we ranked all of the nodes of the graph according to the following criteria:


$distance(C_0, C_i) = median(ShortestPath(C_0, C_i))$

In [27]:
# take the category 0 as input
request_cat = input()
# list of all nodes belong to this category 0
cat_nodes = category_clean[request_cat]

Year_of_birth_unknown


We decided to give the category **'Year_of_birth_unknown'** as input, because it is one of the smallest list of nodes.


In the following code's lines we decided to implement the **Breadth-first search (BFS)**, because it is the algorithm that we thought it could be more efficient and faster than the others. In this way we searched all the shortest paths between each node of the category *'Year_of_birth_unknown'* and all the nodes of the graph. We saved these informations in the dictionary *len_path* that has a graph's node as key and the list of all the distances between the node and all the nodes in the requested category. The distance from the same node it is **0** and, if a node doesn't arrive to the other node, the distance is **infinite**.

In [30]:
# dictionary len_path for key a node as value a dictionary
len_path = {}
for node in all_nodes:
    # this dictionary has as key the starting node and as value a list of all the distances
    len_path[node] = []

In [31]:
def BFS(graph, s, len_path):
    # initialize the distance
    curr_dist = 0
    # update len path the distance of node s 
    len_path[s].append(curr_dist)
    # nodes to visit
    queue = set(graph[s])
    # nodes visited yet
    visited = set([s])  
    # if queue isn't empty
    while queue:
        # update curr_dist 
        curr_dist += 1
        temp = []
        # iterate nodes to visit
        for v in queue:
            # update the dictionary of the shortest distance
            len_path[v].append(curr_dist)
            temp += graph[v]
            visited.add(v)
        # update queue with only the nodes that we haven't visited yet
        queue = (set(temp)).difference(visited)
    return len_path

In [32]:
# iterate the nodes of the category 'Year_of_birth_unknown'
for n in cat_nodes:
    # find all the shortest path
    len_path = BFS(graph,n,len_path)

We decided to save the dictionary *len_path* in a file, because of this algorithm spends much time to work.

In [34]:
# save dictionary
with open('category_shortest_path.pickle','wb') as file:
    pickle.dump(len_path,file, protocol = pickle.HIGHEST_PROTOCOL)

In [36]:
# to open the file with the dictionary
cat_file = open('category_shortest_path.pickle','rb')
shortest_path = pickle.load(cat_file)

At this point we computed $distance(C_0, C_i) = median(ShortestPath(C_0, C_i))$ for each category, according to the category 'Year_of_birth_unknown', our $C_0$.

To save the informations about the median and then create the list of categories, representing the 

$block_{RANKING} =\begin{bmatrix} C_0 \\ C_1 \\ \dots \\ C_c\\ \end{bmatrix}$ , 

we created a dictionary *block_ranking* that has a determinate value of the distance as key and a list of tuples as value. The tuple has two element: the first element is the count of the key in the list and the second is the category $C_i$ that have the $distance(C_0, C_i)$.

In [47]:
# iterate all categories and order for the distance from C0 'Year_of_birth_unknown'
# dictionary that has as key the median of the shortest path and as value a list of the categories
block_ranking = defaultdict(list)
# number of nodes in requested category
cat_nodes = len(category_clean[request_cat])
# iterate all the categories
for cat in category_clean.keys():
    # category requested - median 0
    if cat == request_cat:
        block_ranking[0].append((0,cat))
    # other categories   
    else:
        # number of nodes in the category - cat
        cat_n = len(category_clean[cat])
        # length of the list
        tot = cat_n * cat_nodes
        # intialize list of all the shortest paths of this category - cat
        list_path = []
        # iterate all the node in category cat
        for node in category_clean[cat]:
            # update list
            list_path += shortest_path[node]
        num_inf = tot - (len(list_path))
        # if list smaller than what we expect - append the infinite distances
        if num_inf != 0:   
            list_path += ([np.inf] * (tot - len(list_path)))
        if num_inf > tot//2:
            key = np.inf
            count = num_inf
        else:
            array_path = np.array(list_path)
            # compute the median of this category cat
            key = np.median(array_path)
            count = list_path.count(key)
        value = (count, cat)
        # update the block ranking 
        block_ranking[key].append(value)

In [48]:
print(block_ranking)

defaultdict(<class 'list'>, {inf: [(10435866, 'English_footballers'), (10553063, 'The_Football_League_players'), (7660553, 'Association_football_forwards'), (6061970, 'Association_football_goalkeepers'), (9284793, 'Association_football_midfielders'), (7285457, 'Association_football_defenders'), (464429820, 'Living_people'), (7631868, 'Harvard_University_alumni'), (7401986, 'Major_League_Baseball_pitchers'), (8379594, 'Year_of_death_missing'), (4729434, 'English_cricketers'), (39753055, 'Year_of_birth_missing_(living_people)'), (24219182, 'Main_Belt_asteroids'), (9770594, 'Asteroids_named_for_people'), (4449794, 'Fellows_of_the_Royal_Society'), (8383590, 'Year_of_birth_missing'), (7255785, 'Place_of_birth_missing_(living_people)'), (5148539, 'American_military_personnel_of_World_War_II'), (5116028, 'Windows_games')], 0: [(0, 'Year_of_birth_unknown')], 7.0: [(3156487, 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies'), (3515163, 'Indian_films'), (1060144, 'English_tel

All distances found.

In [49]:
print(list(block_ranking.keys()))

[inf, 0, 7.0, 8.0, 6.0, 9.0]


Then we saved in the list *cat_block* all the categories, ordered according to the distances.

In [50]:
# list of all the categories, ordered according to the distance
cat_block = []
for k in sorted(block_ranking.keys()):
    sort_block =  sorted(block_ranking[k], key=lambda tup: tup[0])
    for cat in sort_block:
        cat_block.append(cat[1])
     
    

This is the $block_{RANKING}$ that we obtained.

In [51]:
print(cat_block)

['Year_of_birth_unknown', 'American_film_actors', 'English-language_films', 'Article_Feedback_Pilot', 'American_Jews', 'English_television_actors', 'British_films', 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 'Indian_films', 'American_television_actors', 'Black-and-white_films', 'American_films', 'People_from_New_York_City', 'English-language_albums', 'Rivers_of_Romania', 'Debut_albums', 'Fellows_of_the_Royal_Society', 'English_cricketers', 'Windows_games', 'American_military_personnel_of_World_War_II', 'Association_football_goalkeepers', 'Place_of_birth_missing_(living_people)', 'Association_football_defenders', 'Major_League_Baseball_pitchers', 'Harvard_University_alumni', 'Association_football_forwards', 'Year_of_death_missing', 'Year_of_birth_missing', 'Association_football_midfielders', 'Asteroids_named_for_people', 'English_footballers', 'The_Football_League_players', 'Main_Belt_asteroids', 'Year_of_birth_missing_(living_people)', 'Living_people']


#### Step 1

We computed subgraph induced by $C_0$ . For each node compute the sum of the weigths of the in-edges.

$score_{article_i} = \sum_{j \in in-edges(article_i)} w_j$ 

In [53]:
# find the score to assign to each node
def score_first_category(C,g):
    # initialize the dictionary of scores assigned to nodes in category C
    global res 
    res = { i : 0 for i in C }
    # for each node in the first category
    for node in C:
        # for each one of its successors (nodes which are reached by an outgoing edge of the previous one) 
        for suc in list(g.successors(node)):
            # if the successor is in the first category
            if suc in C:
                # increase its score by one
                res[suc]+=1
    # update the graph
    for n in res:
        g.node[n]['weight']=res[n]
    # return the updated graph
    return g 

In [54]:
# assign the weigths to the edges
def give_weight_to_edges(C,g):
    # for each node in the considered category
    for n in C:
        # get its score
        node_w=g.node[n]['weight']
        # give a weight to its outgoing edges, based on the node's score
        for e in list(g.out_edges(n)):
            source=e[0]
            target=e[1]
            g[source][target]['weight'] = node_w
    # return the updated graph
    return g

In [55]:
# we have to start giving scores to the nodes of the first category 
res = {}
graph_nx_up = score_first_category(category_clean[cat_block[0]],graph_nx)
graph_nx_up = give_weight_to_edges(category_clean[cat_block[0]],graph_nx_up)

Here we printed the first five nodes's scores of the first category.

In [56]:
for key in list(res.keys())[:5]:
    print(key, res[key])

3335 0
10527 0
16310 0
22286 0
23468 1


#### Step 2

In this step we extended the graph to the nodes that belong to $C_1$ . Thus, for each article in $C_1$ we computed the score as before. Now the in-edges coming from the previous category, $C_0$ , have as weights the score of the node that sends the edge.

In [57]:
# compute the scores for the nodes which are not in the first category
def score_notfirst_category(C,g):
    # compute the scores as in the first category (i.e. taking into account only the nodes in the same category)
    g = score_first_category(C,g)
    # then, for each node in the category
    for n in C:
        # for each one of its incoming edges
        for i in list(g.in_edges(n)):
            # if the edge doesn't come from a node of the same category
            if i[0] not in C:
                # try to update its score, using the weight of the incoming edge
                try:
                    g.node[n]['weight']+=g.node[i[0]]['weight']
                # catch the possible exception (i.e. the edge has not a weight, because the node from which
                # it starts has not been visited yet)
                except:
                    continue
    # return the updated graph
    return g

In [58]:
# we give scores to the nodes of the category C1
graph_nx_up = score_notfirst_category(category_clean[cat_block[1]],graph_nx_up)
graph_nx_up = give_weight_to_edges(category_clean[cat_block[1]],graph_nx_up)

Here we printed the firs five nodes's scores of the category $C_1$, then we assigned the weigths to te edges, according to the weigths of the category $C_0$.

In [59]:
for key in list(res.keys())[:5]:
    print(key, res[key])

174 0
980 0
1088 1
1099 0
1106 1


#### Step 3

Now we repeated *Step 2* up to the last category of the ranking. In the last step we could clearly see the weight update of the edge coming from node E.

In [61]:
# given it we can procede for all the other category, the important things is that we respect the order of the block_ranking
for c in tqdm(cat_block[2:]):
    graph_nx_up = score_notfirst_category(category_clean[c],graph_nx_up)
    graph_nx_up = give_weight_to_edges(category_clean[c],graph_nx_up)

100%|██████████| 33/33 [2:14:46<00:00, 2354.82s/it]


In [67]:
# initialize block vector
block_vector = list()

# for each category 
for category in cat_block:
    # initialize the block of nodes of the category, and turn it into a heap, for faster sorting
    block = list()
    heapq.heapify(block)
    # for each node in the category
    for node in category_clean[category]:
        # push the tuple (score, node) into the heap
        heapq.heappush(block, ((graph_nx_up.node[node]['weight'], node)))
    # get the list of (score, node) tuples from the heap, sorted according to the scores
    block = heapq.nlargest(len(block), block)
    # append it to the block vector
    block_vector.append(block)
        

In [79]:
print(block_vector[0][:100])

[(2629075, 1084068), (1814763, 1639508), (1670602, 158355), (1352982, 1659360), (658894, 1151309), (174077, 1686459), (131765, 1167053), (30420, 1151492), (10402, 1038076), (5044, 1244814), (2345, 233504), (1998, 137662), (1545, 1656777), (704, 1269822), (246, 51844), (220, 1031761), (152, 1113103), (55, 584219), (47, 1659362), (21, 62684), (18, 330031), (15, 528662), (14, 426344), (14, 219346), (13, 1602954), (12, 170163), (11, 836597), (11, 666855), (10, 1656780), (10, 169696), (9, 1656794), (9, 1656455), (9, 672781), (9, 666857), (7, 1343014), (7, 1342864), (7, 170578), (6, 1656778), (6, 1404839), (6, 1404830), (6, 1342960), (6, 666853), (6, 324414), (6, 204079), (6, 78609), (5, 1779656), (5, 1766063), (5, 1203496), (5, 1203235), (5, 1203095), (5, 1109485), (5, 1109348), (5, 672511), (5, 536462), (5, 317555), (5, 174582), (5, 159920), (5, 159730), (5, 159606), (5, 64632), (4, 1765837), (4, 1765831), (4, 1765824), (4, 1602546), (4, 1443739), (4, 1380565), (4, 1344701), (4, 1343206), 

In [77]:
# initialize the final list of ranked articles (coming from the categories, which have also been ranked)
final_rank = []
# for each index (which corresponds to a category), in the block vector
for idx1 in range(len(block_vector)):
    # for each index (which corresponds to a (score, node) tuple), inside the category
    for idx2 in range(len(block_vector[idx1])):
        # get the node from the tuple
        node = block_vector[idx1][idx2][1]
        # append the name of the article (corresponding to the node) to the final list
        final_rank.append(idx_name[node])

In [78]:
print(final_rank[:100])

['Mary Jo Pehl', 'James R. Whelan', 'Michael Munn', 'Frankie Ingrassia', 'Kim Jong-un', 'Ricky Vela', 'Caitlin Dulany', 'Jeryl Prescott', 'Pete Lee-Wilson', 'Jon H. Else', 'Francis Salvador', 'Veronica Vera', 'L Bu', 'Anna Lehr', 'Bradley Burston', 'Jerry Booth', 'D. B. Cooper', 'A. M. Rathnam', 'Bridget Hodson', 'Diogenes Lartius', 'Wilma Subra', 'Gus Kallio', 'Hilde Holovsky', 'Gheorghe tefan', 'Lawrence Rosen (anthropologist)', 'Pausanias (geographer)', 'James Adair (serjeant)', 'Stephen Dingate', 'Dong Zhuo', 'Theocritus', 'Yuan Shu', 'Li Linfu', 'Andrew Gibson (footballer)', 'Tom Faulkner', 'Symeon of Durham', 'Penda of Mercia', 'Stobaeus', 'Zhang Fei', 'William Smith (congressman)', 'Ballard Smith', 'Edwin, Earl of Mercia', 'Joe Harris (cricketer)', 'Chief Crow', 'Tim Stevenson', 'Michael Knighton', 'Stigand', 'Ivan Alexander of Bulgaria', 'Amasis II', 'Horemheb', 'Nefertiti', 'Pope Nicholas III', 'Pope Alexander II', 'Chetan (actor)', 'Sidney Stanley', 'Francis White (Virginia)'

We printed only the first 100 articles in the list *final_rank*.